In [1]:
#imports:
#install tensorflow, keras, pandas, & scikitlearn.
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ###
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix

#from sklearn import metrics
#import matplotlib.pyplot as plt

In [2]:
data_ANN = pd.read_csv('GoldWeekly2001-2021.csv')

data_ANN = data_ANN.drop('Did_Win', axis=1)
data_ANN = data_ANN.drop('Volume', axis=1)

for i in range(len(data_ANN)):
    if data_ANN.iloc[i,-1] == 0:
        continue
    else:
        data_ANN.iloc[i,-1] = data_ANN.iloc[i,-1] / data_ANN.iloc[i,-2] * 100

data_ANN.head(n=4)

,Date,Open,High,Low,Close,Profit
0,1/1/2001,268.399994,268.399994,267.299988,268.000000,-1.529853
1,1/8/2001,268.000000,268.000000,263.899994,263.899994,0.151570
2,1/15/2001,263.299988,264.299988,263.200012,264.299988,-0.567537
3,1/22/2001,266.399994,266.399994,262.799988,262.799988,1.636232


In [3]:
from sklearn import preprocessing

Label = preprocessing.LabelEncoder()

infoFromLabel = Label.fit_transform(data_ANN['Open'])
#infoFromLabel

In [4]:
#Preprocessing data:
data_labels = data_ANN['Date'].copy()
data_ANN = data_ANN.drop('Date', axis = 1)

#Split data for training and testing
X_train, X_Test_Val = train_test_split(data_ANN, test_size = 0.4, train_size = 0.6, shuffle = False)
X_test, X_val = train_test_split(X_Test_Val, test_size = 0.5, train_size = 0.5, shuffle = False)
#Removing the labels from the data.
train_labels = X_train['Profit'].copy()
X_train = X_train.drop('Profit', axis = 1)

test_labels = X_test['Profit'].copy()
X_test = X_test.drop('Profit', axis = 1)

val_labels = X_val['Profit'].copy()
X_val = X_val.drop('Profit', axis = 1)


In [63]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

#Multi-layer-perceptron
#print(X_train)

arr_train_labels = np.array(train_labels)
X_inputA = []
X_inputB = []
X_inputC = []
X_inputD = []

for i in range(len(X_train)):
    X_inputA.append(X_train.iloc[i,0])
    X_inputB.append(X_train.iloc[i,1])
    X_inputC.append(X_train.iloc[i,2])
    X_inputD.append(X_train.iloc[i,3])

X_inputA = np.array(X_inputA)
X_inputB = np.array(X_inputB)
X_inputC = np.array(X_inputC)
X_inputD = np.array(X_inputD)

X_values = [X_inputA, X_inputB, X_inputC, X_inputD]

print(X_train)

openPrice = keras.Input(shape=(len(X_train), 1), name='Open')
highPrice = keras.Input(shape=(len(X_train), 1), name='High')
lowPrice = keras.Input(shape=(len(X_train), 1), name='Low')
closePrice = keras.Input(shape=(len(X_train), 1), name='Close')

openLayer = LSTM(45, return_sequences=False)(openPrice)
highLayer = LSTM(45, return_sequences=False)(highPrice)
lowLayer = LSTM(45, return_sequences=False)(lowPrice)
closeLayer = LSTM(45, return_sequences=False)(closePrice)

output = keras.layers.concatenate([openLayer, highLayer, lowLayer, closeLayer])

output = Dense(10, activation='linear', name='weightedOutput')(output)

print('1')


RNN = Sequential()

#RNN.add.LSTM(LSTM(units=45, activation='relu', return_sequences=False, input_shape = (1, 4)))
RNN.add.LSTM(units=45, input_shape = (1, 1)) #, activation='relu', return_sequences=False))

print('2')
#Dropout regularization is a technique used to avoid overfitting when training neural networks
RNN.add(Dropout(0.2))
RNN.add(LSTM(units = 45, return_sequences = True))
RNN.add(Dropout(0.2))
RNN.add(LSTM(units = 45, return_sequences = True))
RNN.add(Dropout(0.2))
RNN.add(LSTM(units = 45))
RNN.add(Dropout(0.2))

RNN.add(Dense(units = 1))
RNN.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'accuracy')

#tf.keras.backend.set_floatx('float64')
#RNN.fit(X_train, train_labels, epochs = 100, batch_size = 32)
RNN.fit(X_values, arr_train_labels, epochs = 100)

display(RNN)


# MLP = MLPRegressor(hidden_layer_sizes=(5,5,5), max_iter=20_000)
# MLP.fit(X_train, train_labels)

# results = MLP.predict(X_test)

# result_df = pd.DataFrame(X_test)
# result_df = result_df.assign(Profit = test_labels, Predictions = results)

# display(result_df)

            Open         High          Low        Close
0     268.399994   268.399994   267.299988   268.000000
1     268.000000   268.000000   263.899994   263.899994
2     263.299988   264.299988   263.200012   264.299988
3     266.399994   266.399994   262.799988   262.799988
4     262.799988   268.899994   262.799988   267.100006
..           ...          ...          ...          ...
621  1750.300049  1750.500000  1708.400024  1710.900024
622  1714.900024  1721.900024  1684.000000  1704.000000
623  1704.199951  1722.300049  1689.900024  1695.800049
624  1690.500000  1702.800049  1635.800049  1659.099976
625  1657.699951  1666.699951  1653.699951  1654.900024

[626 rows x 4 columns]


NotImplementedError: Cannot convert a symbolic Tensor (lstm_76/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
#currentMoney = 1000

#for ((length of testing data)-1)
#    if predicting_a_win:
#        currentMoney = currentMoney * (Profit[i] + 1)
#    
#print(CurrentMoney) #then do this for validation.
#
#
#
